# Bibliotecas


In [1]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
#from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import TrainingArguments
from transformers import Trainer
import torch
import pandas as pd
from torch.optim import AdamW
from tqdm import tqdm
import time
import numpy as np
from torch import cuda
from sklearn.model_selection import train_test_split
import json

from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models import KeyedVectors

from torch.utils.data import DataLoader

# Funções

In [71]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [72]:
from torch.utils.data import Dataset

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': encoding['input_ids'].squeeze(0)  # Exemplo para MLM
        }

In [73]:
#Dataset do torch auxilia no treinamento dos modelos
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [74]:
"""
This function converts the list of sentences into a BERT input
"""
def bertInput_clean(sentences):

    token_text = "[SEP]".join(sentences)
    
    return token_text


"""
This function converts the list of sentences into a BERT input
"""
def input_clean(sentences):

    token_text = " ".join(sentences)
    
    return token_text

In [75]:
#Obtem os embeddings das sentenças e não das palavras
def executeModel_CLS(model, tokenizer, poi_type):
    
    tokenized_text = tokenizer(poi_type, return_tensors='pt', max_length=512, truncation=True, padding=True)
    
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers = model(**tokenized_text, output_hidden_states = True)
    
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers['hidden_states'], dim=0)
    
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.mean(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec.tolist())
        

    #print ('Shape is: %d x %d' % (len(token_vecs_sum_text_01), len(token_vecs_sum_text_01[0])))
    
    #CLS vector
    return token_vecs_sum[0]

In [76]:
def convertModels(distilbert_model, model_base):
    
     #Obtendo o Índice de cada palavra no modelo
    vocab_tuple = []
    for key in model_base.wv.vocab.keys():
        vocab_tuple.append((key, model_base.wv.vocab[key].index))
    vocab_tuple.sort(key=lambda x:x[1])


    #Construindo os vetores concatenados
    vectors_matrix = []
    for word, index in vocab_tuple:

        try:
            vector = executeModel_CLS(distilbert_model, tokenizer, word)
        except:
            vector = [-1.0]*70

        vectors_matrix.append(np.hstack([vector]).tolist())


    #Convertendo matriz para tipo float32
    vectors_matrix = np.float32(vectors_matrix)

    #Atualizando vetores no modelo base
    model_base.wv.vector_size = len(vectors_matrix[0])
    model_base.vector_size = len(vectors_matrix[0]) 
    model_base.wv.vector = vectors_matrix
    #model_base.wv.syn0 = vectors_matrix
    model_base.wv.vectors = vectors_matrix

    #Não lembro o que essa etapa é
    #model_base.wv.syn0norm = None
    model_base.wv.vectors_norm  = None
    model_base.wv.init_sims()
    
    return model_base

# Realizando todos os passos em Pipeline
Os dados são relações de co-ocorrência binárias entre os tipos do POI central e os tipos dos POIs na vizinhança.

## Versão Traning Validation

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- **O conjunto de validação é o próprio treino**
- **O mascaramento é feito nos tokens correspondentes a toda a palavra**

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

#model_checkpoint = 'roberta-base'
#tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 100
EPOCHS = 15
BATCH_SIZE = 1
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
patience = 3
use_amp = True

osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
for n in range(0, 1):
    for w in np.arange(0.0, 0.1, 0.1):
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-pfp-c'
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)
                #model = RobertaForMaskedLM.from_pretrained(model_checkpoint)
                
                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI (Treino)...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                train_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = sentences[0][3] # => Dado geográfico
                for i, sentence in enumerate(sentences):

                    if((count_sentences % SENTENCE_SIZE) == 0):
                        
                        final_sentence = '[CLS]'.join(local_sentences)
                        train_sentences.append(final_sentence)
                        
                        count_sentences = 1
                        local_sentences = []

                    #Aqui deve ser feito um split_test
                    elif(sentence[3] != actual_type):
                        
                        #Salvando a parte final
                        final_sentence = '[CLS]'.join(local_sentences)
                        train_sentences.append(final_sentence)
                        
                        actual_type = sentence[3]
                        count_sentences = 1
                        
                        #Começando o novo contexto
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences = [bertInput_clean(sentence_text)]

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    train_sentences.append(final_sentence)
                
                print("Gerando conjunto de Treino...")
                #train, validation = train_test_split(train_sentences, test_size=valid_size, random_state=42)
                print('Conjunto de Treino: ', len(train_sentences))

                #Esvaziando memória
                del local_sentences

                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train_sentences, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                #Esvaziando memória
                del train_sentences

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
            
                #Esvaziando memória
                del rand_train, mask_arr_train
                
                #Alterando os tokens de TODA A PALAVRA
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                    
                    for j in selection_train[i]:
                        #Moving front and back
                        b = j - 1
                        f = j + 1

                        #Preenchendo restante do token
                        while ((b > 0) and 
                               (inputs_train.input_ids[i, b] != 101 and 
                                inputs_train.input_ids[i, b] != 102 and 
                                inputs_train.input_ids[i, b] != 103)):

                            inputs_train.input_ids[i, b] = 103
                            b = b-1

                        while ((f < len(inputs_train.input_ids[i])) and 
                               (inputs_train.input_ids[i, f] != 0 and 
                                inputs_train.input_ids[i, f] != 101 and 
                                inputs_train.input_ids[i, f] != 102 and 
                                inputs_train.input_ids[i, f] != 103)):
                            inputs_train.input_ids[i, f] = 103
                            f = f+1
                
                
                    
                #Esvaziando memória
                del selection_train 

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                dataset_val = dataset_train
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=False)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())

                        train_losses.append(loss.item())

        
                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    avg_train_losses.append(train_loss)


                    # early_stopping needs the validation loss to check if it has decresed, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(train_loss, model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        final_time = (time.time() - start_time)
                        has_early_stopping = True
                        training_dictionary = {'epoch': epoch+1,
                                               'epochs': EPOCHS,
                                               'patience': patience,
                                               'train_loss': train_loss,
                                               'avg_train_losses':avg_train_losses,
                                               'time': final_time}
                        break

                    # clear lists to track next epoch
                    train_losses = []

                # load the last checkpoint with the best model
                model.load_state_dict(torch.load('checkpoint.pt'))

                #Novo tempo para o caso das coisas terem sido diferentes
                if(has_early_stopping == False):
                    final_time = (time.time() - start_time)
                    training_dictionary = {'epoch': epoch+1,
                                           'epochs': EPOCHS,
                                           'patience': patience,
                                           'train_loss': train_loss,
                                           'avg_train_losses':avg_train_losses,
                                           'time': final_time}

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_train
                )

                trainer.save_model()
                
                #Salvando as estatísticas do treinamento
                file_name = model_name + '/training_dictionary.json'
                with open(file_name, "w") as outfile:
                    json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del dataset_train
                del train_losses
                del avg_train_losses
                del trainer

## Versão Type Split-I

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- **O conjunto de validação é construído descantando-se os dados replicados**
- **O mascaramento é feito nos tokens correspondentes a toda a palavra**

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

#model_checkpoint = 'roberta-base'
#tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 50
EPOCHS = 15
BATCH_SIZE = 1
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
patience = 3
use_amp = True
weights = [0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]

osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
#osm_tables = ['bins_roads_information', 'bins_lines_information']
for n in range(1, 2):
    #for w in np.arange(0.0, 1.1, 0.1):
    for w in weights:
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-03/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-pfp-c'
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-03/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)
                #model = RobertaForMaskedLM.from_pretrained(model_checkpoint)
                
                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                validation = sentences[['center_poi', 'context_osm']].drop_duplicates()
                validation = validation.values.tolist()
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI (Treino)...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                train_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = sentences[0][3] # => Dado geográfico
                for i, sentence in enumerate(sentences):

                    #Para voltar a versão anterior descomente esse trecho
                    '''if((count_sentences % SENTENCE_SIZE) == 0):
                        
                        final_sentence = '[CLS]'.join(local_sentences)
                        train_sentences.append(final_sentence)
                        
                        count_sentences = 1
                        local_sentences = []'''

                    #Aqui deve ser feito um split_test
                    if(sentence[3] != actual_type):
                        
                        #Esse trecho é novo
                        if(len(local_sentences) >= SENTENCE_SIZE):
                            
                            print('Treino ultrapassou em:', i, 'tam:', len(local_sentences))
                            
                            final_sentence = '[CLS]'.join(local_sentences[0:int(len(local_sentences)/2)])
                            train_sentences.append(final_sentence)
                            
                            final_sentence = '[CLS]'.join(local_sentences[int(len(local_sentences)/2):len(local_sentences)])
                            train_sentences.append(final_sentence)
                        
                        else:
                            #Salvando a parte final
                            final_sentence = '[CLS]'.join(local_sentences)
                            train_sentences.append(final_sentence)

                        actual_type = sentence[3]
                        count_sentences = 1

                        #Começando o novo contexto
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences = [bertInput_clean(sentence_text)]

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    train_sentences.append(final_sentence)
            
                #Esvaziando memória
                del sentences, local_sentences
                
                
                print("Gerando subtextos com foco nos tipos de POI (Validação)...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                val_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = validation[0][1] # => Dado geográfico
                for i, sentence in enumerate(validation):

                    #Descomente para voltar ao original
                    '''if((count_sentences % SENTENCE_SIZE) == 0):
                        
                        final_sentence = '[CLS]'.join(local_sentences)
                        val_sentences.append(final_sentence)
                        
                        count_sentences = 1
                        local_sentences = []'''

                    #Aqui deve ser feito um split_test
                    if(sentence[1] != actual_type):
                        
                        #Esse trecho é novo
                        if(len(local_sentences) >= SENTENCE_SIZE):
                            
                            print('Validação ultrapassou em:', i, 'tam:', len(local_sentences))
                            
                            final_sentence = '[CLS]'.join(local_sentences[0:int(len(local_sentences)/2)])
                            train_sentences.append(final_sentence)
                            
                            final_sentence = '[CLS]'.join(local_sentences[int(len(local_sentences)/2):len(local_sentences)])
                            train_sentences.append(final_sentence)
                        
                        else:
                            #Salvando a parte final
                            final_sentence = '[CLS]'.join(local_sentences)
                            val_sentences.append(final_sentence)
                        
                        actual_type = sentence[1]
                        count_sentences = 1
                        
                        #Começando o novo contexto
                        sentence_text = [sentence[0], sentence[1]]
                        local_sentences = [bertInput_clean(sentence_text)]

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[0], sentence[1]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    val_sentences.append(final_sentence)
            
                #Esvaziando memória
                del validation, local_sentences
                
                print('Conjunto de Treino: ', len(train_sentences))
                print('Conjunto de Validação: ', len(val_sentences))
                
                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train_sentences, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                inputs_val = tokenizer(val_sentences, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_val['labels'] = inputs_val.input_ids.detach().clone()

                #Esvaziando memória
                del train_sentences, val_sentences

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                rand_val = torch.rand(inputs_val.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                mask_arr_val = (rand_val < MASK_PERC) * (inputs_val.input_ids != 101) * \
                           (inputs_val.input_ids != 102) * (inputs_val.input_ids != 0)
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
                selection_val = []
                for i in range(inputs_val.input_ids.shape[0]):
                    selection_val.append(
                        torch.flatten(mask_arr_val[i].nonzero()).tolist()
                    )

                #Esvaziando memória
                del rand_train, mask_arr_train, rand_val, mask_arr_val
                
                #Alterando os tokens de TODA A PALAVRA
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                    
                    for j in selection_train[i]:
                        #Moving front and back
                        b = j - 1
                        f = j + 1

                        #Preenchendo restante do token
                        while ((b > 0) and 
                               (inputs_train.input_ids[i, b] != 101 and 
                                inputs_train.input_ids[i, b] != 102 and 
                                inputs_train.input_ids[i, b] != 103)):

                            inputs_train.input_ids[i, b] = 103
                            b = b-1

                        while ((f < len(inputs_train.input_ids[i])) and 
                               (inputs_train.input_ids[i, f] != 0 and 
                                inputs_train.input_ids[i, f] != 101 and 
                                inputs_train.input_ids[i, f] != 102 and 
                                inputs_train.input_ids[i, f] != 103)):
                            inputs_train.input_ids[i, f] = 103
                            f = f+1
                
                for i in range(inputs_val.input_ids.shape[0]):
                    inputs_val.input_ids[i, selection_val[i]] = 103
                    
                    for j in selection_val[i]:
                        #Moving front and back
                        b = j - 1
                        f = j + 1

                        #Preenchendo restante do token
                        while ((b > 0) and 
                               (inputs_val.input_ids[i, b] != 101 and 
                                inputs_val.input_ids[i, b] != 102 and 
                                inputs_val.input_ids[i, b] != 103)):

                            inputs_val.input_ids[i, b] = 103
                            b = b-1

                        while ((f < len(inputs_val.input_ids[i])) and 
                               (inputs_val.input_ids[i, f] != 0 and 
                                inputs_val.input_ids[i, f] != 101 and 
                                inputs_val.input_ids[i, f] != 102 and 
                                inputs_val.input_ids[i, f] != 103)):
                            inputs_val.input_ids[i, f] = 103
                            f = f+1
                    
                #Esvaziando memória
                del selection_train, selection_val

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                dataset_val = Dataset(inputs_val)
                loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train, inputs_val

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the validation loss as the model trains
                valid_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []
                # to track the average validation loss per epoch as the model trains
                avg_valid_losses = [] 

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=True)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())


                        train_losses.append(loss.item())

                    ######################    
                    # validate the model #
                    ######################
                    model.eval() # prep model for evaluation
                    loop_val = tqdm(loader_val, leave=True)
                    for batch_val in loop_val:

                        # forward pass: compute predicted outputs by passing inputs to the model
                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        labels = batch_val['labels'].to(device)

                        # process
                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        # record validation loss
                        valid_losses.append(loss.item())

                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    valid_loss = np.average(valid_losses)
                    avg_train_losses.append(train_loss)
                    avg_valid_losses.append(valid_loss)

                    # early_stopping needs the validation loss to check if it has decresed, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        final_time = (time.time() - start_time)
                        has_early_stopping = True
                        training_dictionary = {'epoch': epoch+1,
                                               'epochs': EPOCHS,
                                               'patience': patience,
                                               'train_loss': train_loss,
                                               'valid_loss': valid_loss,
                                               'avg_train_losses':avg_train_losses,
                                               'avg_valid_losses': avg_valid_losses,
                                               'time': final_time}
                        break

                    # clear lists to track next epoch
                    train_losses = []
                    valid_losses = []

                # load the last checkpoint with the best model
                model.load_state_dict(torch.load('checkpoint.pt'))

                #Novo tempo para o caso das coisas terem sido diferentes
                if(has_early_stopping == False):
                    final_time = (time.time() - start_time)
                    training_dictionary = {'epoch': epoch+1,
                                           'epochs': EPOCHS,
                                           'patience': patience,
                                           'train_loss': train_loss,
                                           'valid_loss': valid_loss,
                                           'avg_train_losses':avg_train_losses,
                                           'avg_valid_losses': avg_valid_losses,
                                           'time': final_time}

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_val
                )

                trainer.save_model()
                
                #Salvando as estatísticas do treinamento
                file_name = model_name + '/training_dictionary.json'
                with open(file_name, "w") as outfile:
                    json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del loader_val
                del dataset_train
                del dataset_val
                del train_losses
                del valid_losses
                del avg_train_losses
                del avg_valid_losses
                del trainer

Carregando dados...
./geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-1bins_polygons_information-wgt0.2pfp-c.parquet
Quantidade de sentenças: 2452843
Gerando subtextos com foco nos tipos de POI (Treino)...
Treino ultrapassou em: 147 tam: 63
Treino ultrapassou em: 364 tam: 91
Treino ultrapassou em: 519 tam: 65
Treino ultrapassou em: 743 tam: 64
Treino ultrapassou em: 893 tam: 54
Treino ultrapassou em: 1178 tam: 66
Treino ultrapassou em: 1657 tam: 145
Treino ultrapassou em: 1876 tam: 51
Treino ultrapassou em: 1976 tam: 60
Treino ultrapassou em: 2204 tam: 148
Treino ultrapassou em: 2300 tam: 60
Treino ultrapassou em: 2594 tam: 72
Treino ultrapassou em: 2709 tam: 75
Treino ultrapassou em: 2801 tam: 64
Treino ultrapassou em: 2885 tam: 64
Treino ultrapassou em: 2965 tam: 68
Treino ultrapassou em: 3070 tam: 90
Treino ultrapassou em: 3185 tam: 60
Treino ultrapassou em: 3447 tam: 55
Treino ultrapassou em: 3735 tam: 68
Treino ultrapassou em: 3827 tam: 52
Treino ultrapassou em: 4011 tam: 120
Treino 

Treino ultrapassou em: 243173 tam: 182
Treino ultrapassou em: 243288 tam: 65
Treino ultrapassou em: 243376 tam: 76
Treino ultrapassou em: 243501 tam: 90
Treino ultrapassou em: 243854 tam: 78
Treino ultrapassou em: 244002 tam: 52
Treino ultrapassou em: 244523 tam: 72
Treino ultrapassou em: 244731 tam: 136
Treino ultrapassou em: 244876 tam: 65
Treino ultrapassou em: 245075 tam: 52
Treino ultrapassou em: 245244 tam: 169
Treino ultrapassou em: 245313 tam: 51
Treino ultrapassou em: 245513 tam: 60
Treino ultrapassou em: 245747 tam: 135
Treino ultrapassou em: 246002 tam: 91
Treino ultrapassou em: 246210 tam: 96
Treino ultrapassou em: 246372 tam: 72
Treino ultrapassou em: 246487 tam: 80
Treino ultrapassou em: 246683 tam: 85
Treino ultrapassou em: 246897 tam: 112
Treino ultrapassou em: 247163 tam: 70
Treino ultrapassou em: 247499 tam: 130
Treino ultrapassou em: 247724 tam: 144
Treino ultrapassou em: 247793 tam: 51
Treino ultrapassou em: 247859 tam: 51
Treino ultrapassou em: 247991 tam: 102
Trei

Treino ultrapassou em: 486684 tam: 80
Treino ultrapassou em: 486818 tam: 54
Treino ultrapassou em: 486989 tam: 81
Treino ultrapassou em: 487052 tam: 57
Treino ultrapassou em: 487176 tam: 52
Treino ultrapassou em: 487276 tam: 72
Treino ultrapassou em: 487386 tam: 75
Treino ultrapassou em: 487556 tam: 60
Treino ultrapassou em: 487656 tam: 80
Treino ultrapassou em: 487908 tam: 224
Treino ultrapassou em: 487968 tam: 51
Treino ultrapassou em: 488142 tam: 90
Treino ultrapassou em: 488609 tam: 126
Treino ultrapassou em: 488808 tam: 94
Treino ultrapassou em: 488888 tam: 64
Treino ultrapassou em: 489269 tam: 55
Treino ultrapassou em: 489472 tam: 63
Treino ultrapassou em: 489535 tam: 54
Treino ultrapassou em: 489619 tam: 76
Treino ultrapassou em: 489887 tam: 94
Treino ultrapassou em: 490025 tam: 54
Treino ultrapassou em: 490256 tam: 165
Treino ultrapassou em: 490361 tam: 85
Treino ultrapassou em: 490491 tam: 80
Treino ultrapassou em: 490731 tam: 112
Treino ultrapassou em: 490869 tam: 102
Treino 

Treino ultrapassou em: 697546 tam: 65
Treino ultrapassou em: 697762 tam: 56
Treino ultrapassou em: 697828 tam: 57
Treino ultrapassou em: 698179 tam: 182
Treino ultrapassou em: 698283 tam: 56
Treino ultrapassou em: 698481 tam: 63
Treino ultrapassou em: 698754 tam: 66
Treino ultrapassou em: 699157 tam: 56
Treino ultrapassou em: 699609 tam: 112
Treino ultrapassou em: 699871 tam: 70
Treino ultrapassou em: 699959 tam: 64
Treino ultrapassou em: 700239 tam: 69
Treino ultrapassou em: 700533 tam: 72
Treino ultrapassou em: 700946 tam: 170
Treino ultrapassou em: 701076 tam: 55
Treino ultrapassou em: 701286 tam: 75
Treino ultrapassou em: 701493 tam: 144
Treino ultrapassou em: 701598 tam: 80
Treino ultrapassou em: 701876 tam: 68
Treino ultrapassou em: 701992 tam: 52
Treino ultrapassou em: 702420 tam: 50
Treino ultrapassou em: 702552 tam: 90
Treino ultrapassou em: 702757 tam: 195
Treino ultrapassou em: 703025 tam: 112
Treino ultrapassou em: 703341 tam: 128
Treino ultrapassou em: 703407 tam: 51
Trein

Treino ultrapassou em: 921259 tam: 102
Treino ultrapassou em: 921503 tam: 51
Treino ultrapassou em: 921655 tam: 56
Treino ultrapassou em: 921899 tam: 64
Treino ultrapassou em: 922009 tam: 95
Treino ultrapassou em: 922500 tam: 150
Treino ultrapassou em: 922732 tam: 168
Treino ultrapassou em: 923183 tam: 121
Treino ultrapassou em: 923267 tam: 68
Treino ultrapassou em: 923417 tam: 90
Treino ultrapassou em: 923587 tam: 145
Treino ultrapassou em: 923679 tam: 76
Treino ultrapassou em: 923861 tam: 66
Treino ultrapassou em: 924245 tam: 300
Treino ultrapassou em: 924390 tam: 65
Treino ultrapassou em: 924446 tam: 56
Treino ultrapassou em: 924670 tam: 224
Treino ultrapassou em: 924730 tam: 51
Treino ultrapassou em: 924884 tam: 52
Treino ultrapassou em: 924953 tam: 57
Treino ultrapassou em: 925162 tam: 75
Treino ultrapassou em: 925316 tam: 85
Treino ultrapassou em: 925496 tam: 132
Treino ultrapassou em: 925686 tam: 70
Treino ultrapassou em: 925991 tam: 67
Treino ultrapassou em: 926199 tam: 51
Trei

...

Treino ultrapassou em: 2266935 tam: 51
Treino ultrapassou em: 2267085 tam: 108
Treino ultrapassou em: 2267364 tam: 117
Treino ultrapassou em: 2267535 tam: 65
Treino ultrapassou em: 2267598 tam: 60
Treino ultrapassou em: 2267926 tam: 70
Treino ultrapassou em: 2268270 tam: 100
Treino ultrapassou em: 2268502 tam: 98
Treino ultrapassou em: 2268740 tam: 112
Treino ultrapassou em: 2269054 tam: 100
Treino ultrapassou em: 2269288 tam: 96
Treino ultrapassou em: 2269388 tam: 91
Treino ultrapassou em: 2269523 tam: 63
Treino ultrapassou em: 2269749 tam: 127
Treino ultrapassou em: 2269889 tam: 112
Treino ultrapassou em: 2269991 tam: 51
Treino ultrapassou em: 2270239 tam: 50
Treino ultrapassou em: 2270299 tam: 51
Treino ultrapassou em: 2270473 tam: 60
Treino ultrapassou em: 2270697 tam: 96
Treino ultrapassou em: 2270928 tam: 84
Treino ultrapassou em: 2271094 tam: 52
Treino ultrapassou em: 2271334 tam: 192
Treino ultrapassou em: 2271801 tam: 160
Treino ultrapassou em: 2271945 tam: 96
Treino ultrapass

Treino ultrapassou em: 2451209 tam: 55
Treino ultrapassou em: 2451469 tam: 128
Treino ultrapassou em: 2451897 tam: 90
Treino ultrapassou em: 2452065 tam: 128
Treino ultrapassou em: 2452337 tam: 176
Treino ultrapassou em: 2452735 tam: 96
Gerando subtextos com foco nos tipos de POI (Validação)...
Conjunto de Treino:  176216
Conjunto de Validação:  35624
Mascarando dados...
Preparando para o treinamento...


  0%|          | 0/176216 [00:00<?, ?it/s]<ipython-input-4-554098cc9339>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 35624/35624 [04:26<00:00, 133.44it/s]


Validation loss decreased (inf --> 0.028851).  Saving model ...


 56%|█████▋    | 20106/35624 [02:30<01:56, 132.85it/s]

## Versão Type Split-I

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- O conjunto de validação é construído descartando-se os dados replicados

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

#model_checkpoint = 'roberta-base'
#tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 200
EPOCHS = 15
BATCH_SIZE = 1
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
patience = 5
use_amp = True

osm_tables = ['bins_polygons_information', 'bins_points_information']
for n in range(0, 1):
    for w in np.arange(0.0, 0.1, 0.1):
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-01/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-pfp-c'
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT-01/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                #model_name = './geographic/GEOC2VECBERT15TKT-02/austin-sl-tuple-geoc2vec-roberta-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)
                #model = RobertaForMaskedLM.from_pretrained(model_checkpoint)
                
                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                validation = sentences[['center_poi', 'context_osm']].drop_duplicates()
                validation = validation.values.tolist()
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI (Treino)...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                train_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = sentences[0][3] # => Dado geográfico
                for i, sentence in enumerate(sentences):

                    if((count_sentences % SENTENCE_SIZE) == 0):
                        
                        final_sentence = '[CLS]'.join(local_sentences)
                        train_sentences.append(final_sentence)
                        
                        count_sentences = 1
                        local_sentences = []

                    #Aqui deve ser feito um split_test
                    if(sentence[3] != actual_type):
                        
                        #Salvando a parte final
                        final_sentence = '[CLS]'.join(local_sentences)
                        train_sentences.append(final_sentence)
                        
                        actual_type = sentence[3]
                        count_sentences = 1
                        
                        #Começando o novo contexto
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences = [bertInput_clean(sentence_text)]

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    train_sentences.append(final_sentence)
            
                #Esvaziando memória
                del sentences, local_sentences
                
                
                print("Gerando subtextos com foco nos tipos de POI (Validação)...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                val_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = validation[0][1] # => Dado geográfico
                for i, sentence in enumerate(validation):

                    if((count_sentences % SENTENCE_SIZE) == 0):
                        
                        final_sentence = '[CLS]'.join(local_sentences)
                        val_sentences.append(final_sentence)
                        
                        count_sentences = 1
                        local_sentences = []

                    #Aqui deve ser feito um split_test
                    if(sentence[1] != actual_type):
                        
                        #Salvando a parte final
                        final_sentence = '[CLS]'.join(local_sentences)
                        val_sentences.append(final_sentence)
                        
                        actual_type = sentence[1]
                        count_sentences = 1
                        
                        #Começando o novo contexto
                        sentence_text = [sentence[0], sentence[1]]
                        local_sentences = [bertInput_clean(sentence_text)]

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[0], sentence[1]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    val_sentences.append(final_sentence)
            
                #Esvaziando memória
                del validation, local_sentences
                
                print('Conjunto de Treino: ', len(train_sentences))
                print('Conjunto de Validação: ', len(val_sentences))
                

                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train_sentences, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                inputs_val = tokenizer(val_sentences, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_val['labels'] = inputs_val.input_ids.detach().clone()


                #Esvaziando memória
                del train_sentences, val_sentences

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                rand_val = torch.rand(inputs_val.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                mask_arr_val = (rand_val < MASK_PERC) * (inputs_val.input_ids != 101) * \
                           (inputs_val.input_ids != 102) * (inputs_val.input_ids != 0)
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
                selection_val = []
                for i in range(inputs_val.input_ids.shape[0]):
                    selection_val.append(
                        torch.flatten(mask_arr_val[i].nonzero()).tolist()
                    )

                #Esvaziando memória
                del rand_train, mask_arr_train, rand_val, mask_arr_val
                
                #Alterando os tokens
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                
                for i in range(inputs_val.input_ids.shape[0]):
                    inputs_val.input_ids[i, selection_val[i]] = 103
                    
                #Esvaziando memória
                del selection_train, selection_val

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                dataset_val = Dataset(inputs_val)
                loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train, inputs_val

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the validation loss as the model trains
                valid_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []
                # to track the average validation loss per epoch as the model trains
                avg_valid_losses = [] 

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=False)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())


                        train_losses.append(loss.item())

                    ######################    
                    # validate the model #
                    ######################
                    model.eval() # prep model for evaluation
                    loop_val = tqdm(loader_val, leave=True)
                    for batch_val in loop_val:

                        # forward pass: compute predicted outputs by passing inputs to the model
                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        labels = batch_val['labels'].to(device)

                        # process
                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        # record validation loss
                        valid_losses.append(loss.item())

                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    valid_loss = np.average(valid_losses)
                    avg_train_losses.append(train_loss)
                    avg_valid_losses.append(valid_loss)

                    # early_stopping needs the validation loss to check if it has decresed, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        final_time = (time.time() - start_time)
                        has_early_stopping = True
                        training_dictionary = {'epoch': epoch+1,
                                               'epochs': EPOCHS,
                                               'patience': patience,
                                               'train_loss': train_loss,
                                               'valid_loss': valid_loss,
                                               'avg_train_losses':avg_train_losses,
                                               'avg_valid_losses': avg_valid_losses,
                                               'time': final_time}
                        break

                    # clear lists to track next epoch
                    train_losses = []
                    valid_losses = []

                # load the last checkpoint with the best model
                model.load_state_dict(torch.load('checkpoint.pt'))


                #Novo tempo para o caso das coisas terem sido diferentes
                if(has_early_stopping == False):
                    final_time = (time.time() - start_time)
                    training_dictionary = {'epoch': epoch+1,
                                           'epochs': EPOCHS,
                                           'patience': patience,
                                           'train_loss': train_loss,
                                           'valid_loss': valid_loss,
                                           'avg_train_losses':avg_train_losses,
                                           'avg_valid_losses': avg_valid_losses,
                                           'time': final_time}

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_val
                )

                trainer.save_model()
                
                #Salvando as estatísticas do treinamento
                file_name = model_name + '/training_dictionary.json'
                with open(file_name, "w") as outfile:
                    json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del loader_val
                del dataset_train
                del dataset_val
                del train_losses
                del valid_losses
                del avg_train_losses
                del avg_valid_losses
                del trainer

Carregando dados...
./geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-0bins_polygons_information-wgt0.0pfp-c.parquet
Quantidade de sentenças: 2157954
Gerando subtextos com foco nos tipos de POI (Treino)...
Gerando subtextos com foco nos tipos de POI (Validação)...
Conjunto de Treino:  89172
Conjunto de Validação:  20694
Mascarando dados...
Preparando para o treinamento...


  0%|          | 0/89172 [00:00<?, ?it/s]<ipython-input-4-554098cc9339>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2:  78%|███████▊  | 69399/89172 [39:00<11:06, 29.65it/s, loss=0.0109]  

## Versão Type Split-II

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- O conjunto de validação é feito utilizando o split 80-20

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 200
EPOCHS = 15
BATCH_SIZE = 4
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
patience = 5
use_amp = True

osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
for n in range(0, 1):
    for w in np.arange(0.3, 1.1, 0.1):
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKT/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)

                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                bert_sentences = []
                local_sentences = []
                count_sentences = 1
                actual_type = sentences[0][3] # => Dado geográfico
                for i, sentence in enumerate(sentences):

                    if((count_sentences % SENTENCE_SIZE) == 0):
                        count_sentences = 1
                        final_sentence = '[CLS]'.join(local_sentences)
                        bert_sentences.append(final_sentence)
                        local_sentences = []

                    #Aqui deve ser feito um split_test
                    elif(sentence[3] != actual_type):
                        actual_type = sentence[3]
                        count_sentences = 1
                        final_sentence = '[CLS]'.join(local_sentences)
                        bert_sentences.append(final_sentence)
                        local_sentences = []

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    bert_sentences.append(final_sentence)
            
                #Esvaziando memória
                del sentences, local_sentences
                
                print("Gerando conjunto de Treino e Validação...")
                train, validation = train_test_split(bert_sentences, test_size=valid_size, random_state=42)
                print('Conjunto de Treino: ', len(train))
                print('Conjunto de Validação: ', len(validation))
                
                #Esvaziando memória
                del bert_sentences

                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                inputs_val = tokenizer(validation, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_val['labels'] = inputs_val.input_ids.detach().clone()


                #Esvaziando memória
                del train, validation

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                rand_val = torch.rand(inputs_val.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                mask_arr_val = (rand_val < MASK_PERC) * (inputs_val.input_ids != 101) * \
                           (inputs_val.input_ids != 102) * (inputs_val.input_ids != 0)
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
                selection_val = []
                for i in range(inputs_val.input_ids.shape[0]):
                    selection_val.append(
                        torch.flatten(mask_arr_val[i].nonzero()).tolist()
                    )

                #Esvaziando memória
                del rand_train, mask_arr_train, rand_val, mask_arr_val
                
                #Alterando os tokens
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                
                for i in range(inputs_val.input_ids.shape[0]):
                    inputs_val.input_ids[i, selection_val[i]] = 103
                    
                #Esvaziando memória
                del selection_train, selection_val

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                dataset_val = Dataset(inputs_val)
                loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train, inputs_val

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the validation loss as the model trains
                valid_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []
                # to track the average validation loss per epoch as the model trains
                avg_valid_losses = [] 

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=False)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())


                        train_losses.append(loss.item())

                    ######################    
                    # validate the model #
                    ######################
                    model.eval() # prep model for evaluation
                    loop_val = tqdm(loader_val, leave=True)
                    for batch_val in loop_val:

                        # forward pass: compute predicted outputs by passing inputs to the model
                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        labels = batch_val['labels'].to(device)

                        # process
                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        # record validation loss
                        valid_losses.append(loss.item())

                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    valid_loss = np.average(valid_losses)
                    avg_train_losses.append(train_loss)
                    avg_valid_losses.append(valid_loss)

                    # early_stopping needs the validation loss to check if it has decresed, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        final_time = (time.time() - start_time)
                        has_early_stopping = True
                        training_dictionary = {'epoch': epoch+1,
                                               'epochs': EPOCHS,
                                               'patience': patience,
                                               'train_loss': train_loss,
                                               'valid_loss': valid_loss,
                                               'avg_train_losses':avg_train_losses,
                                               'avg_valid_losses': avg_valid_losses,
                                               'time': final_time}
                        break

                    # clear lists to track next epoch
                    train_losses = []
                    valid_losses = []

                # load the last checkpoint with the best model
                model.load_state_dict(torch.load('checkpoint.pt'))


                #Novo tempo para o caso das coisas terem sido diferentes
                if(has_early_stopping == False):
                    final_time = (time.time() - start_time)
                    training_dictionary = {'epoch': epoch+1,
                                           'epochs': EPOCHS,
                                           'patience': patience,
                                           'train_loss': train_loss,
                                           'valid_loss': valid_loss,
                                           'avg_train_losses':avg_train_losses,
                                           'avg_valid_losses': avg_valid_losses,
                                           'time': final_time}

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_val
                )

                trainer.save_model()
                
                #Salvando as estatísticas do treinamento
                file_name = model_name + '/training_dictionary.json'
                with open(file_name, "w") as outfile:
                    json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del loader_val
                del dataset_train
                del dataset_val
                del train_losses
                del valid_losses
                del avg_train_losses
                del avg_valid_losses
                del trainer

## Versão Type Split-II.I

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- O conjunto de validação é feito utilizando o split 80-20
- Não trabalha com Early Stop

In [7]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 200
EPOCHS = 3
BATCH_SIZE = 4
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
patience = 5
use_amp = True

#weights = [0.0, 0.5, 1.0]
#weights = [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]
weights = [0.5]

#osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
osm_tables = ['bins_polygons_information']
for n in range(4, 5):
    
    #for w in np.arange(0.3, 1.1, 0.1):
    for w in weights:
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC PROP/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VEC2BERT15TKT-01/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                
#                 if (osm_table == 'bins_polygons_information'):
#                     file_name = './geographic/GEOC2VEC2/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.csv'
#                 else:
                file_name = './geographic/GEOC2VEC PROP/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VEC2BERT15TKT-01/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)

                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                bert_sentences = []
                local_sentences = []
                actual_type = sentences[0][3] # => Dado geográfico
                for i, sentence in enumerate(sentences):

                    if(sentence[3] != actual_type):
                        
                        final_sentence = '[SEP]'.join(local_sentences)
                        bert_sentences.append(final_sentence)
                        
                        actual_type = sentence[3]
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences = [input_clean(sentence_text)]

                    else:
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(input_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    print(len(local_sentences))
#                     final_sentence = '[SEP]'.join(local_sentences)
#                     print(final_sentence)
                    bert_sentences.append(final_sentence)
            
                #Esvaziando memória
                del sentences, local_sentences
                
                print("Gerando conjunto de Treino e Validação...")
                train, validation = train_test_split(bert_sentences, test_size=valid_size, random_state=42)
                print('Conjunto de Treino: ', len(train))
                print('Conjunto de Validação: ', len(validation))
                
                #print(bert_sentences[0])
                
                #Esvaziando memória
                del bert_sentences

                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding=True)
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                inputs_val = tokenizer(validation, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding=True)
                inputs_val['labels'] = inputs_val.input_ids.detach().clone()


                #Esvaziando memória
                del train, validation

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                rand_val = torch.rand(inputs_val.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                mask_arr_val = (rand_val < MASK_PERC) * (inputs_val.input_ids != 101) * \
                           (inputs_val.input_ids != 102) * (inputs_val.input_ids != 0)
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
                selection_val = []
                for i in range(inputs_val.input_ids.shape[0]):
                    selection_val.append(
                        torch.flatten(mask_arr_val[i].nonzero()).tolist()
                    )

                #Esvaziando memória
                del rand_train, mask_arr_train, rand_val, mask_arr_val
                
                #Alterando os tokens
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                
                for i in range(inputs_val.input_ids.shape[0]):
                    inputs_val.input_ids[i, selection_val[i]] = 103
                    
                #Esvaziando memória
                del selection_train, selection_val

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                dataset_val = Dataset(inputs_val)
                loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train, inputs_val

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the validation loss as the model trains
                valid_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []
                # to track the average validation loss per epoch as the model trains
                avg_valid_losses = [] 

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=False)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())


                        train_losses.append(loss.item())

                    ######################    
                    # validate the model #
                    ######################
                    model.eval() # prep model for evaluation
                    loop_val = tqdm(loader_val, leave=True)
                    for batch_val in loop_val:

                        # forward pass: compute predicted outputs by passing inputs to the model
                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        labels = batch_val['labels'].to(device)

                        # process
                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        # record validation loss
                        valid_losses.append(loss.item())

                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    valid_loss = np.average(valid_losses)
                    avg_train_losses.append(train_loss)
                    avg_valid_losses.append(valid_loss)

                    # clear lists to track next epoch
                    train_losses = []
                    valid_losses = []

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_val
                )

                #trainer.save_model()
                
                final_time = (time.time() - start_time)
#                 training_dictionary = {'epoch': (EPOCHS+1),
#                                        'epochs': EPOCHS,
#                                        'patience': patience,
#                                        'train_loss': train_loss,
#                                        'valid_loss': valid_loss,
#                                        'avg_train_losses':avg_train_losses,
#                                        'avg_valid_losses': avg_valid_losses,
#                                        'time': final_time}
                
#                 #Salvando as estatísticas do treinamento
#                 file_name = model_name + '/training_dictionary.json'
#                 with open(file_name, "w") as outfile:
#                     json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del loader_val
                del dataset_train
                del dataset_val
                del train_losses
                del valid_losses
                del avg_train_losses
                del avg_valid_losses
                del trainer

Carregando dados...
./geographic/GEOC2VEC PROP/austin-sl-tuple-geoc2vec-4bins_polygons_information-wgt0.5pfp-c.parquet
Quantidade de sentenças: 3755228
Gerando subtextos com foco nos tipos de POI...
32
Gerando conjunto de Treino e Validação...
Conjunto de Treino:  350963
Conjunto de Validação:  87741
Mascarando dados...



  0%|          | 0/87741 [00:00<?, ?it/s]/home/salatiel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  

Epoch 0:   0%|          | 0/87741 [00:00<?, ?it/s, loss=16.6]

Preparando para o treinamento...



Epoch 0:   0%|          | 101/87741 [00:02<37:07, 39.34it/s, loss=0.134]


Epoch 0:   0%|          | 193/87741 [00:04<37:02, 39.40it/s, loss=0.0863]


...

Epoch 0:  14%|█▍        | 12133/87741 [05:09<32:04, 39.29it/s, loss=0.00256]


Epoch 0:  14%|█▍        | 12225/87741 [05:11<32:01, 39.30it/s, loss=0.000239]


Epoch 0:  14%|█▍        | 12317/87741 [05:13<31:57, 39.33it/s, loss=0.0102]  


Epoch 0:  14%|█▍        | 12345/87741 [05:14<32:01, 39.25it/s, loss=1.55e-5]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:207                                                                                  │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:341 in step   │
│                                                                                                  │
│   338 │   │                                                                                      │
│   339 │   │   assert len(optimizer_state["found_inf_per_device"]) > 0, "No inf checks were rec   │
│   340 │   │                                                                                      │
│ ❱ 341 │   │   retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)         │
│   342 │   │                                                                                      │
│   343 │   │   optimizer_state["stage"] = OptState.STEPPED                                        │
│   344                                                                                            │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:287 in        │
│ _maybe_opt_step                                                                                  │
│                                                                                                  │
│   284 │                                                                                          │
│   285 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):                │
│   286 │   │   retval = None                                                                      │
│ ❱ 287 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):    │
│   288 │   │   │   retval = optimizer.step(*args, **kwargs)                                       │
│   289 │   │   return retval                                                                      │
│   290                                                                                            │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:287 in        │
│ <genexpr>                                                                                        │
│                                                                                                  │
│   284 │                                                                                          │
│   285 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):                │
│   286 │   │   retval = None                                                                      │
│ ❱ 287 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):    │
│   288 │   │   │   retval = optimizer.step(*args, **kwargs)                                       │
│   289 │   │   return retval                                                                      │
│   290                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [12]:
train_size_avg = []
token_size = []
for tr in train:
    train_size_avg.append(len(tr.split('[SEP]')))
    inputs_train = tokenizer(tr, return_tensors='pt')
    token_size.append(len(inputs_train['input_ids'][0]))

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors


In [13]:
sum(train_size_avg)/len(train_size_avg)

20.69794193154901

In [14]:
sum(token_size)/len(token_size)

221.17695700892162

## Versão POI Split I
- Nesta versão a geração de documentos ocorre de acordo com a mudança de POIs.
- Sempre que ocorre a mudança de POI no conjunto, um novo documento é gerado
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- A separação em treino e validação é feita considerando o split 80-20

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

SENTENCE_SIZE = 200
EPOCHS = 15
BATCH_SIZE = 4
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
patience = 5
use_amp = True

osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
for n in range(0, 1):
    for w in np.arange(0.0, 0.1, 0.1):
        #Ajustando o parâmetro w
        wgt = round(w, 1)
        for osm_table in osm_tables:
        
            #Flag para permitir o treinamento correto
            do_training = False
            
            #Caso especial para carregar os dados de pontos e treinar apenas uma vez
            if(osm_table == 'bins_points_information' and wgt == 0.0):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKP/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-pfp-c'
                do_training = True
                
            elif(osm_table != 'bins_points_information'):
                file_name = './geographic/GEOC2VEC/austin-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
                model_name = './geographic/GEOC2VECBERT15TKP/austin-sl-tuple-geoc2vec-distilbert-MLM-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c'
                do_training = True
                
            if(do_training):
                
                start_time = time.time()
                
                #Carregando o modelo vazio para o finetuning
                model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)

                #Carregar dataset
                print("Carregando dados...")
                print(file_name)
                sentences = pd.read_parquet(file_name)
                sentences = sentences.values.tolist()
                print('Quantidade de sentenças:', len(sentences))


                print("Gerando subtextos com foco nos tipos de POI...")
                #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
                #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
                bert_sentences = []
                local_sentences = []
                actual_id = sentences[0][0]
                count_sentences = 1
                count_append = 0
                for i, sentence in enumerate(sentences):

                    if((count_sentences % SENTENCE_SIZE) == 0):
                        count_sentences = 1
                        final_sentence = '[CLS]'.join(local_sentences)
                        bert_sentences.append(final_sentence)
                        local_sentences = []


                    elif(sentence[0] != actual_id):
                        actual_id = sentence[0]
                        count_sentences = 1
                        final_sentence = '[CLS]'.join(local_sentences)
                        bert_sentences.append(final_sentence)
                        local_sentences = []

                    else:
                        count_sentences+=1
                        sentence_text = [sentence[1], sentence[3]]
                        local_sentences.append(bertInput_clean(sentence_text))

                #Adicionando último trecho
                if(len(local_sentences) > 0):
                    final_sentence = '[CLS]'.join(local_sentences)
                    bert_sentences.append(final_sentence)
            
                #Esvaziando memória
                del sentences, local_sentences
                
                print("Gerando conjunto de Treino e Validação...")
                train, validation = train_test_split(bert_sentences, test_size=valid_size, random_state=42)
                print('Conjunto de Treino: ', len(train))
                print('Conjunto de Validação: ', len(validation))
                
                #Esvaziando memória
                del bert_sentences

                #Tokenizando e salvando uma cópia dos tokens para representar as labels
                inputs_train = tokenizer(train, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_train['labels'] = inputs_train.input_ids.detach().clone()

                inputs_val = tokenizer(validation, return_tensors='pt', max_length=MAX_LEN, truncation = True, padding='max_length')
                inputs_val['labels'] = inputs_val.input_ids.detach().clone()


                #Esvaziando memória
                del train, validation

                #Encontrando MASK_PERC% das frases que serão mascaradas
                #As frases terão a segunda sentença mascarada ex: [CLS]Bar[SEP]Park[SEP] => [CLS]Bar[SEP]#####[SEP]
                print("Mascarando dados...")
                rand_train = torch.rand(inputs_train.input_ids.shape)
                rand_val = torch.rand(inputs_val.input_ids.shape)
                
                #Gerando um mascaramento em posições aleatório
                #101 = [CLS]
                #102 = [SEP]
                mask_arr_train = (rand_train < MASK_PERC) * (inputs_train.input_ids != 101) * \
                           (inputs_train.input_ids != 102) * (inputs_train.input_ids != 0)
                
                mask_arr_val = (rand_val < MASK_PERC) * (inputs_val.input_ids != 101) * \
                           (inputs_val.input_ids != 102) * (inputs_val.input_ids != 0)
                
                #Tokens que terão os tokens mascarados
                selection_train = []
                for i in range(inputs_train.input_ids.shape[0]):
                    selection_train.append(
                        torch.flatten(mask_arr_train[i].nonzero()).tolist()
                    )
                    
                selection_val = []
                for i in range(inputs_val.input_ids.shape[0]):
                    selection_val.append(
                        torch.flatten(mask_arr_val[i].nonzero()).tolist()
                    )

                #Esvaziando memória
                del rand_train, mask_arr_train, rand_val, mask_arr_val
                
                #Alterando os tokens
                #103 = [MASK]
                for i in range(inputs_train.input_ids.shape[0]):
                    inputs_train.input_ids[i, selection_train[i]] = 103
                
                for i in range(inputs_val.input_ids.shape[0]):
                    inputs_val.input_ids[i, selection_val[i]] = 103
                    
                #Esvaziando memória
                del selection_train, selection_val

                #Transformando os dados no objeto dataset do torch
                print("Preparando para o treinamento...")
                dataset_train = Dataset(inputs_train)
                loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
                dataset_val = Dataset(inputs_val)
                loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
                
                #Esvaziando memória
                del inputs_train, inputs_val

                #Preparando o dispotivo para o treinamento
                device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
                torch.cuda.empty_cache()
                # and move our model over to the selected device
                model.to(device)

                optim = AdamW(model.parameters(), lr=LR)

                # to track the training loss as the model trains
                train_losses = []
                # to track the validation loss as the model trains
                valid_losses = []
                # to track the average training loss per epoch as the model trains
                avg_train_losses = []
                # to track the average validation loss per epoch as the model trains
                avg_valid_losses = [] 

                # initialize the early_stopping object
                early_stopping = EarlyStopping(patience=patience, verbose=False)
                scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
                has_early_stopping = False

                for epoch in range(EPOCHS):
                    # setup loop with TQDM and dataloader
                    # activate training mode
                    model.train()
                    loop_train = tqdm(loader_train, leave=True)
                    for batch_train in loop_train:

                        optim.zero_grad()
                        # pull all tensor batches required for training
                        input_ids = batch_train['input_ids'].to(device)
                        attention_mask = batch_train['attention_mask'].to(device)
                        labels = batch_train['labels'].to(device)

                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        scaler.scale(loss).backward()
                        scaler.step(optim)
                        scaler.update()

                        loop_train.set_description(f'Epoch {epoch}')
                        loop_train.set_postfix(loss=loss.item())


                        train_losses.append(loss.item())

                    ######################    
                    # validate the model #
                    ######################
                    model.eval() # prep model for evaluation
                    loop_val = tqdm(loader_val, leave=True)
                    for batch_val in loop_val:

                        # forward pass: compute predicted outputs by passing inputs to the model
                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        labels = batch_val['labels'].to(device)

                        # process
                        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                            outputs = model(input_ids, attention_mask=attention_mask,
                                        labels=labels)
                            loss = outputs.loss

                        # record validation loss
                        valid_losses.append(loss.item())

                    # calculate average loss over an epoch
                    train_loss = np.average(train_losses)
                    valid_loss = np.average(valid_losses)
                    avg_train_losses.append(train_loss)
                    avg_valid_losses.append(valid_loss)

                    # early_stopping needs the validation loss to check if it has decresed, 
                    # and if it has, it will make a checkpoint of the current model
                    early_stopping(valid_loss, model)

                    if early_stopping.early_stop:
                        print("Early stopping")
                        final_time = (time.time() - start_time)
                        has_early_stopping = True
                        training_dictionary = {'epoch': epoch+1,
                                               'epochs': EPOCHS,
                                               'patience': patience,
                                               'train_loss': train_loss,
                                               'valid_loss': valid_loss,
                                               'avg_train_losses':avg_train_losses,
                                               'avg_valid_losses': avg_valid_losses,
                                               'time': final_time}
                        break

                    # clear lists to track next epoch
                    train_losses = []
                    valid_losses = []

                # load the last checkpoint with the best model
                model.load_state_dict(torch.load('checkpoint.pt'))


                #Novo tempo para o caso das coisas terem sido diferentes
                if(has_early_stopping == False):
                    final_time = (time.time() - start_time)
                    training_dictionary = {'epoch': epoch+1,
                                           'epochs': EPOCHS,
                                           'patience': patience,
                                           'train_loss': train_loss,
                                           'valid_loss': valid_loss,
                                           'avg_train_losses':avg_train_losses,
                                           'avg_valid_losses': avg_valid_losses,
                                           'time': final_time}

                #Salvando o modelo pronto
                print("Salvando o modelo...")
                args = TrainingArguments(
                    output_dir=model_name,
                    per_device_train_batch_size=BATCH_SIZE,
                    num_train_epochs=EPOCHS
                )

                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=dataset_train,
                    eval_dataset=dataset_val
                )

                trainer.save_model()
                
                #Salvando as estatísticas do treinamento
                file_name = model_name + '/training_dictionary.json'
                with open(file_name, "w") as outfile:
                    json.dump(training_dictionary, outfile)
                
                #Esvaziando memória
                del loader_train
                del loader_val
                del dataset_train
                del dataset_val
                del train_losses
                del valid_losses
                del avg_train_losses
                del avg_valid_losses
                del trainer

## Versão Type Split-II.II

- Essa versão faz a separação dos dados considerando os tipos de dados geográficos replicados
- Toda vez que o dado geográfico muda, significa que outra sequência de replicação irá iniciar
- Também há um controle de tamanho de sentenças para evitar o estouro da tokenização
- Nesse caso, pequenos documentos com base nessas mudanças são gerados
- O conjunto de validação é feito utilizando o split 80-20
- Não trabalha com Early Stop
- Trabalha com Batches na tokenização para evitar estouros de memória
- Salva os embeddings em Word2Vec para economizar memória

In [9]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

EPOCHS = 2
BATCH_SIZE = 1
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
#patience = 5
use_amp = True
SENTENCE_SIZE = 60
SENTENCE_SIZE_BOUND = 5


osm_tables = ['bins_points_information', 'bins_polygons_information', 'bins_roads_information', 'bins_lines_information']
#osm_tables = ['bins_polygons_information', 'bins_roads_information', 'bins_lines_information']

for n in range(7, 9, 1):
    
    #Ajustando o parâmetro w
    for osm_table in osm_tables:

        #Caso especial para carregar os dados de pontos e treinar apenas uma vez
        if(osm_table == 'bins_points_information'):
            
            #file_name = './Atlanta/GeoContext2Vec - μ36/atlanta-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
            #model_name_W2V = './Atlanta/GeoContext2Vec W2V - μ36/atlanta-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.model'
            #model_name_DW2V = './Atlanta/GeoContext2Vec DISTILW2V - μ36/atlanta-sl-tuple-geoc2vec-distilw2v-' + str(n) + osm_table + '-pfp-c.model'
            
            file_name = './Boston/GeoContext2Vec - μ55/boston-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.parquet'
            model_name_W2V = './Boston/GeoContext2Vec W2V - μ55/boston-sl-tuple-geoc2vec-' + str(n) + osm_table + '-pfp-c.model'
            model_name_DW2V = './Boston/GeoContext2Vec DISTILW2V - μ55/boston-sl-tuple-geoc2vec-distilw2v-' + str(n) + osm_table + '-pfp-c.model'

        else:
            
            #wgt = 0.9
            #file_name = './Atlanta/GeoContext2Vec - μ36/atlanta-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
            #model_name_W2V = './Atlanta/GeoContext2Vec W2V - μ36/atlanta-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.model'
            #model_name_DW2V = './Atlanta/GeoContext2Vec DISTILW2V - μ36/atlanta-sl-tuple-geoc2vec-distilw2v-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c.model'
            
            wgt = 0.0
            file_name = './Boston/GeoContext2Vec - μ55/boston-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.parquet'
            model_name_W2V = './Boston/GeoContext2Vec W2V - μ55/boston-sl-tuple-geoc2vec-' + str(n) + osm_table + '-wgt' + str(wgt) + 'pfp-c.model'
            model_name_DW2V = './Boston/GeoContext2Vec DISTILW2V - μ55/boston-sl-tuple-geoc2vec-distilw2v-' + str(n) + osm_table + '-wgt' + str(wgt) + '-pfp-c.model'


        #Carregando o modelo vazio para o finetuning
        model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)

        #Carregar dataset
        print("Carregando dados...")
        print(file_name)
        sentences = pd.read_parquet(file_name)
        sentences = sentences.values.tolist()
        print('Quantidade de sentenças:', len(sentences))


        print("Gerando subtextos com foco nos tipos de POI...")
        #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
        #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
        bert_sentences = []
        local_sentences = []
        count_sentences = 1
        actual_type = sentences[0][3] # => Dado geográfico
        for i, sentence in enumerate(sentences):

            if((count_sentences % SENTENCE_SIZE) == 0):
                count_sentences = 1
                final_sentence = '[SEP]'.join(local_sentences)
                bert_sentences.append(final_sentence)
                local_sentences = []

            #Aqui deve ser feito um split_test
            elif(sentence[3] != actual_type):
                actual_type = sentence[3]
                count_sentences = 1
                final_sentence = '[SEP]'.join(local_sentences)
                if len(local_sentences) >= SENTENCE_SIZE_BOUND:
                    bert_sentences.append(final_sentence)
                local_sentences = []

            else:
                count_sentences+=1
                sentence_text = [sentence[1], sentence[3]]
                local_sentences.append(bertInput_clean(sentence_text))
                #local_sentences.append(input_clean(sentence_text))

        #Adicionando último trecho
        if(len(local_sentences) > 0):
            final_sentence = '[SEP]'.join(local_sentences)
            bert_sentences.append(final_sentence)


        print(bert_sentences[0])
        #Esvaziando memória
        del sentences, local_sentences

        print("Gerando dataset de treinamento...")
        print("Quantidade de textos:", len(bert_sentences))

        # Define os datasets
        train_dataset = TextDataset(bert_sentences, tokenizer, max_len=MAX_LEN)
        #Esvaziando memória
        del bert_sentences


        # Cria DataLoaders para batches
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        #Esvaziando memória
        #del train_dataset#, val_dataset


        #Preparando o dispotivo para o treinamento
        #Carregando o modelo vazio para o finetuning
        model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)
        device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
        torch.cuda.empty_cache()
        # and move our model over to the selected device
        model.to(device)

        optim = AdamW(model.parameters(), lr=LR)


        # initialize the early_stopping object
        #early_stopping = EarlyStopping(patience=patience, verbose=False)
        scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
        has_early_stopping = False

        for epoch in range(EPOCHS):
            # setup loop with TQDM and dataloader
            # activate training mode
            model.train()
            loop_train = tqdm(train_loader, leave=True)
            for batch_train in loop_train:


                # Inputs do batch
                input_ids = batch_train["input_ids"].squeeze(1)  # Remove dimensões extras
                attention_mask = batch_train["attention_mask"].squeeze(1)
                labels = batch_train["labels"].squeeze(1)

                # Esvaziando memória
                #del batch_train

                # Encontrando MASK_PERC% das frases que serão mascaradas
                rand = torch.rand(input_ids.shape)

                # Máscara aleatória baseada no MASK_PERC
                mask_arr = (rand < MASK_PERC) * (input_ids != 101) * \
                           (input_ids != 102) * (input_ids != 0)

                # Encontrando as posições a serem mascaradas
                selection = []
                for i in range(input_ids.shape[0]):
                    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

                # Alterando os tokens para [MASK] (103)
                for i in range(input_ids.shape[0]):
                    input_ids[i, selection[i]] = 103

                optim.zero_grad()
                # pull all tensor batches required for training
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                    outputs = model(input_ids, attention_mask=attention_mask,
                                labels=labels)
                    loss = outputs.loss

                scaler.scale(loss).backward()
                scaler.step(optim)
                scaler.update()

                loop_train.set_description(f'Epoch {epoch}')
                loop_train.set_postfix(loss=loss.item())


        #Salvando o modelo pronto
        print("Salvando o modelo em W2V...")

        #Carregando Modelos e base
        model_base = Word2Vec.load(model_name_W2V)
        model.to('cpu')
        model.eval()
        model_base = convertModels(model, model_base)

        print('saving file:', model_name_DW2V)
        model_base.save(model_name_DW2V)

        #Esvaziando memória
        del train_loader
        del model_base
        del model

Carregando dados...
./Boston/GeoContext2Vec - μ55/boston-sl-tuple-geoc2vec-7bins_polygons_information-wgt0.0pfp-c.parquet
Quantidade de sentenças: 4785397
Gerando subtextos com foco nos tipos de POI...
Shopping[SEP]polygons_amenity_parking[SEP]Shopping[SEP]polygons_amenity_parking[SEP]Shopping Centers[SEP]polygons_amenity_parking[SEP]Shopping Centers[SEP]polygons_amenity_parking[SEP]Shopping Centers[SEP]polygons_amenity_parking
Gerando dataset de treinamento...
Quantidade de textos: 243560


Epoch 0:   0%|          | 71/243560 [00:01<1:01:03, 66.47it/s, loss=0.131] 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:175                                                                                  │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:341 in step   │
│                                                                                                  │
│   338 │   │                                                                                      │
│   339 │   │   assert len(optimizer_state["found_inf_per_device"]) > 0, "No inf checks were rec   │
│   340 │   │                                                                                      │
│ ❱ 341 │   │   retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)         │
│   342 │   │                                                                                      │
│   343 │   │   optimizer_state["stage"] = OptState.STEPPED                                        │
│   344                                                                                            │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:287 in        │
│ _maybe_opt_step                                                                                  │
│                                                                                                  │
│   284 │                                                                                          │
│   285 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):                │
│   286 │   │   retval = None                                                                      │
│ ❱ 287 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):    │
│   288 │   │   │   retval = optimizer.step(*args, **kwargs)                                       │
│   289 │   │   return retval                                                                      │
│   290                                                                                            │
│                                                                                                  │
│ /home/salatiel/anaconda3/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:287 in        │
│ <genexpr>                                                                                        │
│                                                                                                  │
│   284 │                                                                                          │
│   285 │   def _maybe_opt_step(self, optimizer, optimizer_state, *args, **kwargs):                │
│   286 │   │   retval = None                                                                      │
│ ❱ 287 │   │   if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):    │
│   288 │   │   │   retval = optimizer.step(*args, **kwargs)                                       │
│   289 │   │   return retval                                                                      │
│   290                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

# II.II DIOGO

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint)

EPOCHS = 2
BATCH_SIZE = 1
MAX_LEN = 512
MASK_PERC = 0.15
LR = 5e-5
valid_size = 0.2
#patience = 5
use_amp = True
SENTENCE_SIZE = 60
SENTENCE_SIZE_BOUND = 5


osm_tables = ['points_information', 'polygons_information', 'roads_information', 'lines_information']

    
#Ajustando o parâmetro w
for osm_table in osm_tables:

    file_name = './Models/new-york-sl-tuple-geoc2vec-μ90-pois_'+ osm_table +'-pfp-c.csv'
    model_name_W2V = './Models/new-york-sl-tuple-geoc2vec-μ90-pois_' + osm_table + '-pfp-c.model'
    model_name_DW2V = './Models/new-york-sl-tuple-geoc2vec-μ90-pois-distilw2v-' + osm_table + '-pfp-c.model'

    #Carregando o modelo vazio para o finetuning
    model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)

    #Carregar dataset
    print("Carregando dados...")
    print(file_name)
    sentences = pd.read_csv(file_name)
    sentences = sentences.values.tolist()
    print('Quantidade de sentenças:', len(sentences))


    print("Gerando subtextos com foco nos tipos de POI...")
    #Criando textos menores com o conjunto de duas palavras dos POIs e dados geográficos
    #Esse método considera a mudança do TIPO de dado geográfico para criar um novo subtexto
    bert_sentences = []
    local_sentences = []
    count_sentences = 1
    actual_type = sentences[0][1] # => Dado geográfico
    for i, sentence in enumerate(sentences):

        if((count_sentences % SENTENCE_SIZE) == 0):
            count_sentences = 1
            final_sentence = '[SEP]'.join(local_sentences)
            bert_sentences.append(final_sentence)
            local_sentences = []

        #Aqui deve ser feito um split_test
        elif(sentence[1] != actual_type):
            actual_type = sentence[1]
            count_sentences = 1
            final_sentence = '[SEP]'.join(local_sentences)
            if len(local_sentences) >= SENTENCE_SIZE_BOUND:
                bert_sentences.append(final_sentence)
            local_sentences = []

        else:
            count_sentences+=1
            sentence_text = [sentence[1], sentence[2]]
            local_sentences.append(bertInput_clean(sentence_text))
            #local_sentences.append(input_clean(sentence_text))

    #Adicionando último trecho
    if(len(local_sentences) > 0):
        final_sentence = '[SEP]'.join(local_sentences)
        bert_sentences.append(final_sentence)


    print(bert_sentences[0])
    #Esvaziando memória
    del sentences, local_sentences

    print("Gerando dataset de treinamento...")
    print("Quantidade de textos:", len(bert_sentences))

    # Define os datasets
    train_dataset = TextDataset(bert_sentences, tokenizer, max_len=MAX_LEN)
    #Esvaziando memória
    del bert_sentences


    # Cria DataLoaders para batches
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    #Esvaziando memória
    #del train_dataset#, val_dataset


    #Preparando o dispotivo para o treinamento
    #Carregando o modelo vazio para o finetuning
    model = DistilBertForMaskedLM.from_pretrained(model_checkpoint)
    device = 'cuda' if cuda.is_available() else 'cpu' # CPU OR GPU
    torch.cuda.empty_cache()
    # and move our model over to the selected device
    model.to(device)

    optim = AdamW(model.parameters(), lr=LR)


    # initialize the early_stopping object
    #early_stopping = EarlyStopping(patience=patience, verbose=False)
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    has_early_stopping = False

    for epoch in range(EPOCHS):
        # setup loop with TQDM and dataloader
        # activate training mode
        model.train()
        loop_train = tqdm(train_loader, leave=True)
        for batch_train in loop_train:


            # Inputs do batch
            input_ids = batch_train["input_ids"].squeeze(1)  # Remove dimensões extras
            attention_mask = batch_train["attention_mask"].squeeze(1)
            labels = batch_train["labels"].squeeze(1)

            # Esvaziando memória
            #del batch_train

            # Encontrando MASK_PERC% das frases que serão mascaradas
            rand = torch.rand(input_ids.shape)

            # Máscara aleatória baseada no MASK_PERC
            mask_arr = (rand < MASK_PERC) * (input_ids != 101) * \
                       (input_ids != 102) * (input_ids != 0)

            # Encontrando as posições a serem mascaradas
            selection = []
            for i in range(input_ids.shape[0]):
                selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

            # Alterando os tokens para [MASK] (103)
            for i in range(input_ids.shape[0]):
                input_ids[i, selection[i]] = 103

            optim.zero_grad()
            # pull all tensor batches required for training
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                outputs = model(input_ids, attention_mask=attention_mask,
                            labels=labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optim)
            scaler.update()

            loop_train.set_description(f'Epoch {epoch}')
            loop_train.set_postfix(loss=loss.item())


    #Salvando o modelo pronto
    print("Salvando o modelo em W2V...")

    #Carregando Modelos e base
    model_base = Word2Vec.load(model_name_W2V)
    model.to('cpu')
    model.eval()
    model_base = convertModels(model, model_base)

    print('saving file:', model_name_DW2V)
    model_base.save(model_name_DW2V)

    #Esvaziando memória
    del train_loader
    del model_base
    del model

Carregando dados...
./Models/new-york-sl-tuple-geoc2vec-μ90-pois_points_information-pfp-c.csv
Quantidade de sentenças: 423061
Gerando subtextos com foco nos tipos de POI...
Arts & Crafts Store[SEP]points_access_yes[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_bicycle_parking[SEP]Arts & Crafts Store[SEP]points_amenity_post_box[SEP]Arts & Crafts Store[SEP]points_amenity_post_box[SEP]Arts & Crafts Store[SEP]points_amenity_pub[SEP]Arts & Crafts Store[SEP]points_amenity_waste_basket[SEP]Arts & Crafts Store[SEP]points_amenity_waste_basket[SEP]Arts & Crafts Store[SEP]points_amenity_waste_b

Epoch 0:  41%|████▏     | 3502/8453 [02:03<02:58, 27.81it/s, loss=6.86e-5] 

In [2]:
#model_base_dsb = Word2Vec.load('./Models/new-york-sl-tuple-geoc2vec-pois-distilw2v-lines_information-pfp-c.model')
#model_base_dsb = Word2Vec.load('./Models/new-york-sl-tuple-geoc2vec-pois-distilw2v-points_information-pfp-c.model')
model_base_dsb = Word2Vec.load('./model/Bert/new-york-sl-tuple-geoc2vec-μ90-pois-distilw2v-roads_information-pfp-c.model')
#model_base_dsb = Word2Vec.load('./Models/new-york-sl-tuple-geoc2vec-pois-distilw2v-roads_information-pfp-c.model')

In [3]:
model_base_dsb.wv.most_similar("Cemetery")

[('Library', 0.9755752682685852),
 ('Synagogue', 0.9755641222000122),
 ('Church', 0.9725342392921448),
 ('Museum', 0.9717111587524414),
 ('Historic Site', 0.9714446663856506),
 ('Park', 0.9705073833465576),
 ('Playground', 0.9699022173881531),
 ('Restaurant', 0.9697355031967163),
 ('Hotel', 0.9683917760848999),
 ('Campground', 0.9678623080253601)]

In [8]:
import numpy as np

In [10]:
words = list(model_base_dsb.wv.vocab.keys())
word_vectors = np.array([model_base_dsb.wv[word] for word in words])
word_vectors.shape

(281, 768)

In [55]:
for value in model_base_dsb.wv.vocab:
    if("_" not in value):
        print(value)

Arts & Crafts Store
Bridge
Home (private)
Mobile Phone Shop
Food & Drink Shop
Ferry
Office
Music Venue
Subway
Park
Burger Joint
Bank
Coffee Shop
Clothing Store
Electronics Store
Deli / Bodega
University
Bus Station
Light Rail
Mexican Restaurant
Other Great Outdoors
Neighborhood
Ice Cream Shop
American Restaurant
BBQ Joint
Other Nightlife
Building
College Academic Building
Train Station
Department Store
Tanning Salon
Gym / Fitness Center
Medical Center
Factory
School
Airport
Hotel
Mall
Fast Food Restaurant
Hardware Store
Road
Bar
Campground
Cupcake Shop
Gas Station / Garage
General Travel
Residential Building (Apartment / Condo)
Laundry Service
Chinese Restaurant
Toy / Game Store
Government Building
Convention Center
Drugstore / Pharmacy
Community College
Bagel Shop
Vegetarian / Vegan Restaurant
Candy Store
Miscellaneous Shop
Pizza Place
General College & University
Movie Theater
Sandwich Place
Scenic Lookout
Stadium
Housing Development
Theater
Synagogue
General Entertainment
German Res

In [62]:
model_base_w2v = Word2Vec.load('./Models/new-york-sl-tuple-geoc2vec-pois_polygons_information-pfp-c.model')

In [68]:
model_base_w2v.wv.most_similar("Church")

[('Movie Theater', 0.9996915459632874),
 ('American Restaurant', 0.9996589422225952),
 ('Gas Station / Garage', 0.9996418356895447),
 ('Athletic & Sport', 0.9996417760848999),
 ('Zoo', 0.9996351003646851),
 ('polygons_sport_horse_racing', 0.99962317943573),
 ('Student Center', 0.9996138215065002),
 ('polygons_building_bridge', 0.9996100664138794),
 ('Other Nightlife', 0.9996029734611511),
 ('Flea Market', 0.9996013641357422)]